# APIs Profiling 

In [1]:
%pip install git+https://github.com/c3di/im2im.git

  Cloning https://github.com/c3di/im2im.git to /tmp/pip-req-build-q_0st31v
  Running command git clone --filter=blob:none --quiet https://github.com/c3di/im2im.git /tmp/pip-req-build-q_0st31v
  Resolved https://github.com/c3di/im2im.git to commit 74ab3ae85d3ad2eb1c8cc37f1c0bcbad3e5d82e7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install py-cpuinfo
%pip install gputil

import cpuinfo
import GPUtil

cpu_name = cpuinfo.get_cpu_info()['brand_raw']
print(f"CPU: {cpu_name}")

gpus = GPUtil.getGPUs()
print(f"GPU:0: {GPUtil.getGPUs()[0].name}")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
CPU: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz
GPU:0: NVIDIA GeForce GTX 1080 Ti


## im2im

In [3]:
from im2im import _code_generator

def clear_cache_in_code_generator():
    _code_generator._cache = {}

In [12]:
import cProfile
import pstats
import numpy as np
from im2im import im2im, im2im_code, Image

clear_cache_in_code_generator()

image_size = (2000, 2000, 3)
image = np.random.randint(0, 256, image_size, dtype=np.uint8)
print("from image:", type(image), image.shape, image.dtype)

with cProfile.Profile() as profile:
    to_be_converted = Image(image, "numpy.rgb_uint8")
    converted:Image = im2im(to_be_converted, "torch.gpu")

    results = pstats.Stats(profile)

    im = converted.raw_image

print("to image:", type(im), im.shape, im.dtype, im.device)
print(f"took {results.total_tt} seconds")

print(f'\nThe conversion code is:\n{im2im_code("to_be_converted", to_be_converted.metadata, "converted", converted.metadata)}')

from image: <class 'numpy.ndarray'> (2000, 2000, 3) uint8
to image: <class 'torch.Tensor'> torch.Size([1, 3, 2000, 2000]) torch.float32 cuda:0
took 0.031001577999999995 seconds

The conversion code is:
import torch
image = torch.from_numpy(to_be_converted)
image = image.permute(2, 0, 1)
image = image.unsqueeze(0)
image = image / 255.0
converted = image.cuda()


## im2im_code

In [14]:
from im2im import im2im_code, get_possible_metadata, find_target_metadata

clear_cache_in_code_generator()

with cProfile.Profile() as profile:
    # the metadata can be found via preset or from image.metadata like previous example
    source = get_possible_metadata("numpy.rgb_uint8")
    target = find_target_metadata(source, "torch.gpu")
    code = im2im_code("to_be_converted", source, "converted", target)

    results = pstats.Stats(profile)
    
print(f'The conversion code is:\n\n{code}')
print(f"\nGenerating such code took {results.total_tt} seconds")

The conversion code is:

import torch
image = torch.from_numpy(to_be_converted)
image = image.permute(2, 0, 1)
image = image.unsqueeze(0)
image = image / 255.0
converted = image.cuda()

Generating such code took 0.0008920029999999999 seconds
